In [1]:
# Set up code checking
# Set up filepaths
import os
os.chdir(os.path.join(os.path.expanduser('~'), 'kaggle'))

In [3]:
import numpy as np
import pandas as pd


## Load data into dataframes

In [6]:
train_path = os.path.join('data', 'house-price-data', 'train.csv')
train_data = pd.read_csv(train_path)

test_path = os.path.join('data', 'house-price-data', 'test.csv')
test_data = pd.read_csv(test_path)